# Models (PyTorch)
模型的創建和使用。通常使用`AutoModel`，從檢查點實例化模型時。類別`AutoModel`及其所有相關類別實際上是模型庫中可用的各種模型的簡單包裝。可以自動猜測適合的模型架構，然後使用該架構實例化模型。

如果知道要使用的模型類型，可以直接使用定義其架構的類別。以`BERT`模型bf

安裝需要的套件

In [1]:
!pip install datasets evaluate transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 18.6 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


初始化BERT模型，載入設定的類別，並且載入模型

In [2]:
from transformers import BertConfig, BertModel

# 建立config
config = BertConfig()

# 從config建立模型
model = BertModel(config)

其中的一些屬性：屬性`hidden_size`定義向量的大小，`num_hidden_layers`定義 Transformer 模型的層數

In [3]:
print(config)

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.40.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



### 隨機初始化的hidden states
此狀態下模型可以使用，但會輸出亂碼；需要先經過訓練。可以針對手上的任務從頭開始訓練模型，但需要很長時間和大量資料，為了避免不必要的重複工作，必須能夠分享和重複使用別人已經訓練過的模型，會將初始化參數導入

In [4]:
from transformers import BertConfig, BertModel

config = BertConfig()
model = BertModel(config)

# Model is randomly initialized!

使用`from_pretrained()`方法。其中的名稱必須是huggingface model中的完整名稱，或是一個擁有別人模型完整資料的目錄

In [5]:
from transformers import BertModel

model = BertModel.from_pretrained("bert-base-cased")

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

### 說明
可以替換`BertModel`為`AutoModel`類別。建議使用`AutoModel`，會自動偵測需要的架構及初始值，不再需要自己設定。如果要針對同類任務套用不同的模型，不需要自己寫載入程式。


在上面的程式碼範例中沒有使用`BertConfig`，而是透過`bert-base-cased`標識符載入預訓練模型。這是BERT作者自己訓練的模型checkpoint。該模型現在使用checkpoint的所有權重進行初始化。它可以直接用於對其所訓練的任務進行推理，也可以對新任務進行微調。透過使用預先訓練的權重進行訓練而不是從頭開始，我們可以快速取得良好的結果。


權重已被下載並快取`from_pretrained(`)在快取資料夾中（因此以後對該方法的呼叫不會重新下載它們），預設為`~/.cache/huggingface/transformers`。您可以透過設定環境變數來自訂快取資料夾`HF_HOME`。


用於載入模型的標識符可以是模型中心上任何模型的模型，只要它與 BERT 架構相容即可。

儲存模型在本機電腦

In [6]:
model.save_pretrained("./bert_model")

### 說明
會產生兩個檔案，分別是模型架構的`config.json`和預訓練的權重`model.safetenors`

也可以直接印出模型長相

In [ ]:
print(model)

模型只能處理數字——tokenizer產生的數字。但在我們討論標記器之前，讓我們先探討一下模型接受哪些輸入。tokenizer可以負責將輸入轉換為適當的框架的張量

In [8]:
sequences = ["Hello!", "Cool.", "Nice!"]

In [10]:
encoded_sequences = [
    [101, 7592, 999, 102],
    [101, 4658, 1012, 102],
    [101, 3835, 999, 102],
]

這是一個編碼序列的列表：列表的列表。張量只接受矩形形狀（矩陣）「矩陣」已經是矩形形狀，因此將其轉換為張量很容易：

In [11]:
import torch

model_inputs = torch.tensor(encoded_sequences)

In [13]:
output = model(model_inputs)

In [14]:
print(output)

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 4.4496e-01,  4.8276e-01,  2.7797e-01,  ..., -5.4032e-02,
           3.9393e-01, -9.4770e-02],
         [ 2.4943e-01, -4.4093e-01,  8.1772e-01,  ..., -3.1917e-01,
           2.2992e-01, -4.1172e-02],
         [ 1.3668e-01,  2.2518e-01,  1.4502e-01,  ..., -4.6915e-02,
           2.8224e-01,  7.5566e-02],
         [ 1.1789e+00,  1.6738e-01, -1.8187e-01,  ...,  2.4671e-01,
           1.0441e+00, -6.1974e-03]],

        [[ 3.6436e-01,  3.2464e-02,  2.0258e-01,  ...,  6.0110e-02,
           3.2451e-01, -2.0996e-02],
         [ 7.1866e-01, -4.8725e-01,  5.1740e-01,  ..., -4.4012e-01,
           1.4553e-01, -3.7545e-02],
         [ 3.3223e-01, -2.3271e-01,  9.4876e-02,  ..., -2.5268e-01,
           3.2172e-01,  8.1076e-04],
         [ 1.2523e+00,  3.5754e-01, -5.1321e-02,  ..., -3.7840e-01,
           1.0526e+00, -5.6255e-01]],

        [[ 2.4042e-01,  1.4718e-01,  1.2110e-01,  ...,  7.6062e-02,
           3.3564e-01,  2